In [1]:
# document checker
# author: sizhong du
# since: 2025-03-10


# doc_path = r'./files/正式招标文件.docx'
# doc_path = r'./files/1. 河北省材料采购招标文件示范文本(试行).docx'
# doc_path = './files/（定稿）河北税务2025年税务专网线路租用采购招标文件.doc'
# doc_path = './files/招标文件-唐山乐亭绿色交通车储一体化储能电站项目设备采购（定稿）.docx'
doc_path = './files/test.docx'


# load docx
from docx import Document as WordDocument
def load_docx(doc_path):
    doc = WordDocument(doc_path)
    tables = doc.tables
    table_idx = 0
    full_text = []
    for element in doc.element.body:
        if element.tag.endswith('p'):
            para_text = element.text.strip()
            full_text.append(para_text)
        elif element.tag.endswith('tbl'):
            if table_idx < len(tables):
                for row in tables[table_idx].rows:
                    row_content = [cell.text.strip() if cell.text else '' for cell in row.cells]
                    row_text = '\t'.join(row_content)
                    full_text.append(row_text)
                    table_idx += 1
    return '\n'.join(full_text)
    # return full_text

full_text = load_docx(doc_path)
print('full_text', len(full_text))

# from langchain_community.document_loaders import UnstructuredWordDocumentLoader
# loader = UnstructuredWordDocumentLoader(doc_path)
# docs = loader.load()
# print(docs)
# print(len(docs[0].page_content))


# split text
from langchain_text_splitters import TokenTextSplitter
def split_text(text):
    text_splitter = TokenTextSplitter(
        chunk_size=2000,
        chunk_overlap=100,
    )
    splited_text = text_splitter.split_text(text)
    return splited_text

# from langchain.text_splitter import RecursiveCharacterTextSplitter
# def split_text2(docs):
#     text_splitter = RecursiveCharacterTextSplitter(
#         chunk_size=1000,
#         chunk_overlap=200,
#         length_function=len,
#         separators=["\n\n## ", "\n\n", "\n", " "]
#     )
#     splited_docs = text_splitter.split_documents(docs)
#     return splited_docs

# splited_docs = split_text2(docs)
# print('splited_docs', len(splited_docs))


# embedding
from langchain_community.embeddings import DashScopeEmbeddings
embeddings = DashScopeEmbeddings(
    model="text-embedding-v2",
    dashscope_api_key="sk-a5c5eb662fa64bb0b50e25765808d9f1"
)
# from langchain_ollama import OllamaEmbeddings
# embeddings = OllamaEmbeddings(
#     model="bge-m3:latest",
# )


# vector store
# vector_store = InMemoryVectorStore(embedding=embeddings)
# vector_store.add_documents(documents=splited_docs)
# retriever = vector_store.as_retriever(
#     search_type="mmr",
#     search_kwargs={"k": 1, "fetch_k": 2, "lambda_mult": 0.5},
# )
# retriever.invoke("开标时间")


# llm init
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    model = "qwen2.5:7b",
    # model = "deepseek-r1:14b",
    base_url = "http://192.168.31.5:11434/v1",
    api_key = "ollama",
    temperature = 0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)
# result = llm.invoke("你是谁？")
# print(result)

template = """基于以下上下文：
{context}

问题：{question}
"""
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(template)

# retrieve chain
# from langchain_core.runnables import RunnablePassthrough
# from langchain_core.output_parsers import StrOutputParser
# chain = (
#     {"context": full_text, "question": RunnablePassthrough()}
#     | prompt
#     | llm
#     | StrOutputParser()
# )
# response = chain.invoke("文件发售截止时间是多少？")
# print(response)


# 定义提示词模板
# 章节检查
chapter_check_template = """
请根据以下步骤审核目标文档，确认其是否完整包含所有指定章节：

## 1. 指定章节清单‌
请依次检查文档中是否存在以下章节（严格匹配标题名称及顺序）：
- [第一章 招标公告]（例：第一章 招标公告）
- [第二章 投标人须知]（例：第二章 投标人须知）
- [第三章 评标办法]（例：第三章　评标办法（综合评估法））
- [第四章 合同条款及格式]（例：第四章 合同条款及格式）

## 2. 检查标准‌
- 章节标题需准确无误，层级清晰（如1.1、2.3.4等编号符合逻辑）
- 章节内容完整，无空标题或占位符
- 若文档包含额外章节，不影响审核结果，但需在备注中说明

## 3. 输出格式要求‌
以Markdown表格形式反馈结果，包含：
| 指定章节 | 是否存在（是/否）| 实际位置（页码/章节号）| 备注 |
| - | - | - | - |
|（示例：）| - | - | - |		
| 第三章 评标办法 | 是 | 第2页（1.1）| 标题未加粗 |

## 4. 最终结论：
- 若所有指定章节完整存在‌：通过‌
- 若缺失≥1章节：不通过‌，需在备注中标注缺失项

## 目标文档：{text}
"""

prompt = ChatPromptTemplate.from_template(chapter_check_template)

from langchain_core.runnables import RunnablePassthrough
chain = (
    {"text": RunnablePassthrough()}
    | prompt
    | llm
)
response = chain.invoke(full_text)
print(response)



full_text 1895
content='根据提供的文档内容，我们进行逐项检查并反馈结果如下：\n\n| 指定章节 | 是否存在（是/否）| 实际位置（页码/章节号）| 备注 |\n| - | - | - | - |\n| 第一章 招标公告 | 是 | 3-12页（1.1-1.4） | 标题未加粗 |\n| 第二章 投标人须知 | 是 | 13-15页（2.0） | 无 |\n| 第三章 评标办法 | 是 | 16-17页（3.0） | 无 |\n| 第四章 合同条款及格式 | 否 | - | 无 |\n\n## 最终结论：\n文档中缺失第四章合同条款及格式，因此审核结果为不通过。\n\n请补充第四章合同条款及格式的相关内容。' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 198, 'prompt_tokens': 1630, 'total_tokens': 1828, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen2.5:7b', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'stop', 'logprobs': None} id='run-70776559-8d52-47cd-8f20-f97c3ee3212d-0' usage_metadata={'input_tokens': 1630, 'output_tokens': 198, 'total_tokens': 1828, 'input_token_details': {}, 'output_token_details': {}}


full_text 1895
content='根据提供的文档内容，我们进行逐项检查并反馈结果如下：\n\n| 指定章节 | 是否存在（是/否）| 实际位置（页码/章节号）| 备注 |\n| - | - | - | - |\n| 第一章 招标公告 | 是 | 3-12页（1.1-1.4） | 标题未加粗 |\n| 第二章 投标人须知 | 是 | 13-15页（2.0） | 无 |\n| 第三章 评标办法 | 是 | 16-17页（3.0） | 无 |\n| 第四章 合同条款及格式 | 否 | - | 无 |\n\n## 最终结论：\n文档中缺失第四章合同条款及格式，因此审核结果为不通过。\n\n请补充第四章合同条款及格式的相关内容。' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 198, 'prompt_tokens': 1630, 'total_tokens': 1828, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen2.5:7b', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'stop', 'logprobs': None} id='run-aaad2fe3-b21d-473a-b83e-980ef112744f-0' usage_metadata={'input_tokens': 1630, 'output_tokens': 198, 'total_tokens': 1828, 'input_token_details': {}, 'output_token_details': {}}

### 提示词模板
---

请根据以下步骤审核目标文档，确认其是否完整包含所有指定章节：

## 1. 指定章节清单‌
请依次检查文档中是否存在以下章节（严格匹配标题名称及顺序）：
- [第一章 招标公告]（例：第一章 招标公告）
- [第二章 投标人须知]（例：第二章 投标人须知）
- [第三章 评标办法]（例：第三章　评标办法（综合评估法））
- [第四章 合同条款及格式]（例：第四章 合同条款及格式）

## 2. 检查标准‌
- 章节标题需准确无误，层级清晰（如1.1、2.3.4等编号符合逻辑）
- 章节内容完整，无空标题或占位符
- 若文档包含额外章节，不影响审核结果，但需在备注中说明

## 输出格式要求‌
以Markdown表格形式反馈结果，包含：
| 指定章节 | 是否存在（是/否）| 实际位置（页码/章节号）| 备注 |
| - | - | - | - |
|（示例：）| - | - | - |		
| 第三章 评标办法 | 是 | 第2页（1.1）| 标题未加粗 |

## 最终结论：
- 若所有指定章节完整存在‌：通过‌
- 若缺失≥1章节：不通过‌，需在备注中标注缺失项